In [1]:
# pip install transformers datasets

     ---------------------------------------- 0.0/44.1 kB ? eta -:--:--
     -------------------------- ----------- 30.7/44.1 kB 640.0 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/44.1 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 44.1/44.1 kB 360.6 kB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.6 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 764.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/69.9 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/69.9 kB ? eta -:--:--
     ---------------- --------------------- 30.7/69.9 kB 640.0 kB/s eta 0:00:01
     --------------------------------- ---- 61.4/69.9 kB 544.7 kB/s eta 0:00:01


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [2]:
# import matplotlib.pyplot as plt
# import seaborn as sns


# # Compute lengths
# article_lengths = [len(article.split()) for article in dataset['train']['article']]
# summary_lengths = [len(summary.split()) for summary in dataset['train']['highlights']]

# # Plotting Article Lengths
# plt.figure(figsize=(14, 6))
# sns.histplot(article_lengths, bins=50, color='blue')
# plt.title('Distribution of Article Lengths')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

# # Plotting Summary Lengths
# plt.figure(figsize=(14, 6))
# sns.histplot(summary_lengths, bins=100, color='red')
# plt.title('Distribution of Summary Lengths')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

In [3]:
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# # Join all summaries
# text = " ".join(summary for summary in dataset['train']['highlights'])

# # Optionally, you can clean the text (remove stopwords, punctuation, etc.)
# # For example:
# # text = clean_text_function(text)

# # Generate word cloud
# wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# # Plot the word cloud
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')  # Disable axis
# plt.show()


In [4]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('lucadiliello/bart-small')
model = BartForConditionalGeneration.from_pretrained('lucadiliello/bart-small', ignore_mismatched_sizes=True)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


In [5]:
def preprocess_function(examples):
    # Tokenize the articles and summaries (highlights)
    inputs = tokenizer(examples['article'], padding='max_length', truncation=True, max_length=512)
    labels = tokenizer(examples['highlights'], padding='max_length', truncation=True, max_length=150)

    # Ensure that labels are correctly aligned with the input format
    inputs['labels'] = labels['input_ids']
    return inputs


# Apply the preprocessing function to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [6]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {device}")

Using device: cuda


In [7]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(  
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',
    report_to="none",
    save_steps=500,
    save_total_limit=3, 
    remove_unused_columns=False,
    load_best_model_at_end=True,
    eval_strategy="steps",
    eval_steps=500,
)

# Initialize Trainer
trainer = Trainer(
    model=model.to('cuda:0'),
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

In [8]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
c:\Users\vishn\OneDrive\Desktop\GUI\creategui\Lib\site-packages\transformers\trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have fu

  0%|          | 0/215337 [00:00<?, ?it/s]

c:\Users\vishn\OneDrive\Desktop\GUI\creategui\Lib\site-packages\transformers\trainer.py:3033: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = torch.load

{'loss': 1.0632, 'grad_norm': 3.8183319568634033, 'learning_rate': 3.859600534358607e-05, 'epoch': 0.69}


  0%|          | 0/3342 [00:00<?, ?it/s]

...

{'eval_loss': 1.077158808708191, 'eval_runtime': 515.8402, 'eval_samples_per_second': 25.915, 'eval_steps_per_second': 6.479, 'epoch': 0.75}
{'loss': 1.0683, 'grad_norm': 3.2717361450195312, 'learning_rate': 3.743233241946219e-05, 'epoch': 0.76}


  0%|          | 0/3342 [00:00<?, ?it/s]

{'eval_loss': 1.0713622570037842, 'eval_runtime': 559.522, 'eval_samples_per_second': 23.892, 'eval_steps_per_second': 5.973, 'epoch': 0.76}
{'loss': 1.0689, 'grad_norm': 2.749483346939087, 'learning_rate': 3.731596512704981e-05, 'epoch': 0.77}


  0%|          | 0/3342 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(model.config.vocab_size)

50264


In [34]:

# Tokenize a sample input
inputs = tokenizer("Hello, how are you?", return_tensors="pt")
print("Input IDs:", inputs['input_ids'])

# Check token IDs range
vocab_size = tokenizer.vocab_size
print("Vocab Size:", vocab_size)
if (inputs['input_ids'] >= vocab_size).any():
    raise ValueError("Some token IDs are out of range")

Input IDs: tensor([[    0, 31414,     6,   141,    32,    47,   116,     2]])
Vocab Size: 50265
